In [1]:
pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import numpy as np
from pathlib import Path
import torch.nn as nn
import torchvision
from torchvision.io import read_image

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.act1  = nn.ReLU()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
       
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0)
        self.act2  = nn.ReLU()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.fc1   = nn.Linear(5 * 5 * 16, 120)
        self.act3  = nn.ReLU()
        
        self.fc2   = nn.Linear(120, 84)
        self.act4  = nn.Tanh()
        
        self.fc3   = nn.Linear(84, 10)
    
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        x = self.fc1(x)
        x = self.act3(x)
        x = self.fc2(x)
        x = self.act4(x)
        x = self.fc3(x)
        
        return x

In [3]:
net = Net()

In [4]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)

In [5]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from random import randint
from torchvision.transforms import Resize
from PIL import Image
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        full_list = list(Path(img_dir).glob("*.jpg"))
        # transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((180, 320))])
        # for img in full_list[:512]:
        # image = np.asarray(Image.open(img).convert('RGB'))
        # image = Image.open(img)
        # print(type(image))
        # resized_img = transform(image)
        self.images = full_list[:512]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = read_image(str(self.images[idx]))
        label = randint(0, 4)
        if self.transform:
            transform = Resize(size=(180, 320))
            image = self.transform(image)
        return image, label

In [33]:
pathname = "unique_uis/combined"
ds = CustomImageDataset(pathname)

from torch.utils.data import random_split

train_size = int(0.8 * len(ds))
test_size = len(ds) - train_size
train, test = random_split(ds, [train_size, test_size])

from torch.utils.data import DataLoader

train_loader = DataLoader(train, batch_size=2, shuffle=True)

test_loader = DataLoader(test, batch_size=2, shuffle=False)

# for i, batch in enumerate(train_loader):
# print(i, batch)

In [34]:
net.train()

for epoch_index in range(3):
    for batch, label in train_loader:
        optimizer.zero_grad()
        pred = net(batch)
        loss_val = loss(pred, label)
        loss_val.backward()
        optimizer.step()

RuntimeError: Input type (unsigned char) and bias type (float) should be the same